# CDR Experiments via Pymagicc
##### CC-BY 4.0 2019 @safiume

In [ ]:
import expectexception
import numpy as np
import pandas as pd
from datetime import datetime
from matplotlib import pyplot as plt
%matplotlib inline
from os import listdir
from os.path import join, dirname
from pprint import pprint

In [ ]:
import pymagicc
from pymagicc import MAGICC6
from pymagicc.io import MAGICCData, read_cfg_file, NoReaderWriterError, join_timeseries
from pymagicc.scenarios import ( rcp26, rcp45, rcps )
print(pymagicc.__version__)

In [ ]:
## list declines to run
declines = {"emax": "", "lmax": "", "emin": "", "lmin": ""}
SCEN_DIR = "SCEN"
MAGICC6_DIR = "CDRRUN"

##Start of Magic settings
magcfg = { "startyear" : 1765,
          "endyear" : 2500,
          "core_climatesensitivity" : 3,
          "co2_tempfeedback_yrstart" : 2005,
          "co2_fertilization_yrstart" : 2005,
          "co2_switchfromconc2emis_year" : 2005,
          "rf_mhalo_constantafteryr" : 2500,
          "rf_fgas_constantafteryr" : 2500 }
### End MAGICC Settings

In [ ]:
## CDRex emission declines
def read_declines (names):
    for name in (names):
        declinefile = join(SCEN_DIR, "ONC" + name + ".SCEN")
        mdata = MAGICCData(declinefile,
                           columns={"model": ["CDRex"], "scenario": [name]}
                          )
        declines[name] = mdata
    return declines

In [ ]:
### Run Declines
print(declines.keys())
declinedata = read_declines(declines.keys())
print(declinedata["emax"]["variable"][0])
print(declines["lmin"]["time"].tail(1))

## Scenarios

The four RCP scenarios are already preloaded as DataFrames in pyhector. They contain the following emissions:


In [ ]:
rcp26.head()

In [ ]:
declines["emax"].tail(2)

They are split up in regions:

In [ ]:
rcp26["region"].unique()

In [ ]:
pprint(rcps.timeseries().groupby(["scenario"]))
rcps["scenario"].unique()

They have the following units:

In [ ]:
#pymagicc.units
rcp26[["variable", "unit"]].drop_duplicates().head(2)

A plot of four categories in RCP3PD

In [ ]:
categories_to_plot = [
    "Emissions|" + v 
    for v in ["CO2|MAGICC Fossil and Industrial", "CO2|MAGICC AFOLU", "CH4", "N2O"]
]
rcp26.filter(
    variable = categories_to_plot
).pivot_table(
    index = "time", 
    columns = ["variable", "unit", "region"],
    aggfunc = "sum"
).groupby(level = "variable", axis = 1).plot(figsize = (12, 7));

Fossil fuel emissions for the four RCP scenarios.

In [ ]:
declines["emax"]["unit"][0]

In [ ]:
fig, (ax1,ax2) = plt.subplots(2,1, figsize = (16, 10))

for name, scen in declines.items():
    scen.filter(
        variable = ["Emissions|CO2|MAGICC Fossil and Industrial", 
                    "Emissions|CO2|MAGICC AFOLU"],
        region = "World",
    ).line_plot(ax = ax1);
    ax1.set_ylabel(scen["unit"][0])
    ax1.set_title(
        "OpenCDR RPC experimental scenarios for Fossil Fuel emissions (CO₂)")
    ax1.set_xlim([datetime(2000, 1, 1), datetime(2500, 1, 1)])
    scen.filter(
        variable = ["Emissions|CH4", "Emissions|CO"],
        region = "World",
    ).line_plot(ax = ax2);
    ax2.set_ylabel(scen["unit"][3])
    ax2.set_title(
        "OpenCDR RPC experimental scenarios for Methane & Carbon Monoxide (CH4, CO)")
    ax2.set_xlim([datetime(2000, 1, 1), datetime(2500, 1, 1)])

## Running MAGICC

A single `pymagicc` run doesn't take long and returns a Pandas Dict.
(If not on Windows, the very first run might be slower due to setting up Wine.)

In [ ]:
# NBVAL_IGNORE_OUTPUT
%time results = pymagicc.run(rcp26, **magcfg)
results = ""
%time results = pymagicc.run(declines["emin"], **magcfg)
results = ""
%time results = pymagicc.run(rcp26, **magcfg)
slice = results[["scenario","region","variable", "unit"]]
results = ""
print(slice[slice["region"].isin(["World"])])

In [ ]:
splt1 = {"surftemp":"Surface Temperature"}
splt2 = {"rf":"Radiative Forcing"}
splt3 = {"emiss":"*Emissions|CO2*"}
splt4 = {"conc":"*Concentrations|CO2"}

fig, (ax1,ax2,ax3,ax4) = plt.subplots(4, 1, figsize=(24, 32))

def displaymg (run, axes, variable):
    run.filter(
        variable=variable,
        region="World"
    ).line_plot(ax=axes, x="time");
    return(run.filter)

def populateplts (raw):
    displaymg(raw, ax1, splt1["surftemp"])
    displaymg(raw, ax2, splt2["rf"])
    displaymg(raw, ax3, splt3["emiss"])
    displaymg(raw, ax4, splt4["conc"])

def lableplts ():
    ax1.set_ylabel("ºC")
    ax1.set_title(splt1["surftemp"])
    ax1.set_xlim([datetime(2000, 1, 1), datetime(magcfg["endyear"], 1, 1)])
    ax2.set_ylabel("Wm⁻²")
    ax2.set_title(splt2["rf"])
    ax2.set_xlim([datetime(2000, 1, 1), datetime(magcfg["endyear"], 1, 1)])
    ax3.set_ylabel("GtC")
    ax3.set_title(splt3["emiss"])
    ax3.set_xlim([datetime(2000, 1, 1), datetime(magcfg["endyear"], 1, 1)])
    ax4.set_ylabel("ppm")
    ax4.set_title(splt4["conc"])
    ax4.set_xlim([datetime(2000, 1, 1), datetime(magcfg["endyear"], 1, 1)])

with MAGICC6() as magicc:
    for name, sdf in rcps.timeseries().groupby(["scenario"]):
        raw = pymagicc.run(MAGICCData(sdf.copy()), **magcfg)
        cntpltsA = populateplts(raw)
    for name, scen in declines.items():
        raw = pymagicc.run(scen, **magcfg)
        cntpltsB = populateplts(raw)
    if (cntpltsA == cntpltsB):
        lableplts()


The default parameters are the ones that were used to produce the RCP GHG concentrations (see also http://live.magicc.org/). Of course it's easy to change them.

In [ ]:
low = pymagicc.run(rcp45, core_climatesensitivity=1.5, endyear=2500)
default = pymagicc.run(rcp45, core_climatesensitivity=3, endyear=2500)
high = pymagicc.run(rcp45, core_climatesensitivity=4.5, endyear=2500)

In [ ]:
filtering = {
    "variable": "Surface Temperature",
    "region": "World",
    "year": range(1850, magcfg["endyear"]),
}

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(16, 9))
default.filter(**filtering).line_plot(x="time", ax=ax)
plt.fill_between(
    low.filter(**filtering)["time"].values,
    low.filter(**filtering).timeseries().values.squeeze(),
    high.filter(**filtering).timeseries().values.squeeze(),
    color="lightgray"
)

plt.title(
    "RCP 4.5 with equilibrium climate sensitivity set to 1.5, 3, and 4.5"
)
plt.ylabel("°C");